In [1]:
from icecream import ic
import scipy.stats
import pandas as pd
import numpy as np
from pandarallel import pandarallel
from warnings import warn
pandarallel.initialize()

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
input_dir = "/mnt/datadisk/data/Projects/water/inputs/"
overwrite_pqt = True
dataset_type = 'HR'  #, 'BR', 'CR', 'HW', 'IR', 'KR', 'MR', 'PR', 'AR'] ??['AN', 'FC, 'FP', 'HW', 'PV', 'SC'] < 25 surveys
# 'BR' (birth R), 'HW!' (WHO height/weight standards children), 'IR'/'PR'? (Individual R, hh member R,), 'KR' (children R), 'AR' (HIV R)

preprocessing_f = f"{input_dir}Preprocessing_df_V3_{dataset_type}_unified.pkl"
out_f = f"{input_dir}Manually_adjusted_df_V3_{dataset_type}.pkl"

In [3]:
df = pd.read_pickle(preprocessing_f)

In [4]:
for col in df.columns.tolist():
    print(col)
print(len(df['generated clnr from case id'].dropna()))
print(len(df['generated hhnr from case id'].dropna()))
df

country code and phase
GEID
DHSID
DHSID + HHID
version_nr
subversion_nr
generated hhnr from case id
generated clnr from case id
cluster number
household number
number of household members
month of interview
year of interview
day of interview
type of place of residence
case identification
source of drinking water
has electricity
has radio
has bicycle
has car/truck
owns sheep
region
type of toilet facility
has refrigerator
number of eligible children for height and weight
translator used
has telephone (land-line)
has bank account
total adults measured
language of questionnaire
share toilet with other households
location of source for water
anything done to water to make safe to drink
water usually treated by: boil
water usually treated by: add bleach/chlorine
number of mosquito bed nets
native language of respondent
household selected for domestic violence module
place where household members wash their hands
presence of water at hand washing place
items present: soap or detergent
items 

,country code and phase,GEID,DHSID,DHSID + HHID,version_nr,subversion_nr,generated hhnr from case id,generated clnr from case id,cluster number,household number,...,has television,owns goats,has motorcycle/scooter,water usually treated by: use water filter,owns horses/ donkeys/ mules,time to get to water source (minutes),owns cows/ bulls,"owns livestock, herds or farm animals",owns chickens/poultry,country code
0,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000001,5,1,NaN,NaN,1,1,...,yes,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,AO
1,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000002,5,1,NaN,NaN,1,2,...,yes,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,AO
2,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000003,5,1,NaN,NaN,1,3,...,yes,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,AO
3,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000004,5,1,NaN,NaN,1,4,...,yes,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,AO
4,AO5,AOGE52FL00000001,AOHR51SV00000001,AOHR51SV0000000100000005,5,1,NaN,NaN,1,5,...,yes,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,AO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853038,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000025,7,2,NaN,NaN,400,25,...,no,7.0,no,no,0,0.0,NaN,yes,31.0,ZW
1853039,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000026,7,2,NaN,NaN,400,26,...,no,0,no,no,0,0.0,NaN,yes,10.0,ZW
1853040,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000027,7,2,NaN,NaN,400,27,...,no,3.0,no,no,0,10.0,NaN,yes,0,ZW
1853041,ZW7,ZWGE72FL00000400,ZWHR72SV00000400,ZWHR72SV0000040000000028,7,2,NaN,NaN,400,28,...,no,8.0,no,no,0,0.0,NaN,yes,5.0,ZW


In [5]:
# value counts/amount of households seems valid
df[df['generated clnr from case id'].notna()][['GEID', 'cluster number']].value_counts()

GEID              cluster number
CIGE81FL00000190  190               27
CIGE81FL00000356  356               26
CIGE81FL00000380  380               25
CIGE81FL00000496  496               24
CIGE81FL00000359  359               24
                                    ..
CIGE81FL00000270  270               15
CIGE81FL00000013  13                14
CIGE81FL00000030  30                14
CIGE81FL00000060  60                14
CIGE81FL00000016  16                 9
Name: count, Length: 539, dtype: int64

In [6]:
# Correct water related values
print(df[['source of drinking water']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['time to get to water source (minutes)']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['location of source for water']].value_counts(dropna=False, normalize=True).head(50) *100)

print(df[['source of drinking water', 'location of source for water']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['time to get to water source (minutes)', 'location of source for water']].value_counts(dropna=False, normalize=True).head(50) *100)
df[['source of drinking water', 'time to get to water source (minutes)', 'location of source for water']].value_counts(dropna=False, normalize=True).head(50) *100

source of drinking water                            
public tap/standpipe                                    14.657350
tube well or borehole                                   14.249103
unprotected well                                        13.624185
piped into dwelling                                     11.699621
river/dam/lake/ponds/stream/canal/irrigation channel    11.263581
protected well                                           8.785387
piped to yard/plot                                       8.305744
unprotected spring                                       5.730952
protected spring                                         3.716805
piped to neighbor                                        2.272047
sachet water                                             1.493597
rainwater                                                1.196356
well in residence                                        1.164139
water seller                                             1.112171
other                  

source of drinking water                              time to get to water source (minutes)  location of source for water
piped into dwelling                                   0.0                                    NaN                             6.567306
piped to yard/plot                                    0.0                                    NaN                             6.314910
piped into dwelling                                   NaN                                    NaN                             5.095349
tube well or borehole                                 10.0                                   elsewhere                       2.048684
public tap/standpipe                                  10.0                                   elsewhere                       1.889109
tube well or borehole                                 30.0                                   elsewhere                       1.645725
piped to neighbor                                     0.0                 

In [7]:
def correct_water(row):
    src = row['source of drinking water']
    loc = row['location of source for water']
    time = row['time to get to water source (minutes)']
    if pd.isna(src):
        return row
    if pd.notna(loc) and pd.notna(time):
        return row
    if pd.isna(loc):
        names = ['dwelling', 'residence']
        if any([dw in src for dw in names]):
            row['location of source for water'] = 'in own dwelling'
        names = ['yard', 'plot', 'courtyard', 'compound']
        if any([dw in src for dw in names]):
            row['location of source for water'] = 'in own yard/plot'
        names = ['neighbor', 'public', 'water seller', 'sachet water', 'river']
        if any([dw in src for dw in names]):
            row['location of source for water'] = 'elsewhere'
            
        # last check depending on time
        if pd.notna(time) and pd.isna(row['location of source for water']):
            if np.isreal(time):
                if time > 1:
                    row['location of source for water'] = 'elsewhere'
                else:
                    row['location of source for water'] = 'in own yard/plot'
                if time == 0 and row['location of source for water'] == 'in own yard/plot':
                    row['time to get to water source (minutes)'] = 1.0
            else:
                print(time, type(time))
        
        # assume it is in own yard/plot because it is nan now 
        # (gets skipped if it is in own dwelling or own yard/plot, these are mostly wells/springs so yard/plot is more likely)
        if pd.isna(row['location of source for water']):
            row['location of source for water'] = 'in own yard/plot'
            
    loc = row['location of source for water']
    if pd.isna(time):
        if loc == 'in own dwelling':
            row['time to get to water source (minutes)'] = 0.0
        elif loc == 'in own yard/plot':
            row['time to get to water source (minutes)'] = 1.0
        # elif loc == 'elsewhere':
        #     row['time to get to water source (minutes)'] = 5
    return row

df['time to get to water source (minutes)'] = df['time to get to water source (minutes)'].astype('Float32')

df2 = df.apply(correct_water, axis=1)
df = df2
# Correct water related values
print(df[['source of drinking water']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['time to get to water source (minutes)']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['location of source for water']].value_counts(dropna=False, normalize=True).head(50) *100)

print(df[['source of drinking water', 'location of source for water']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['time to get to water source (minutes)', 'location of source for water']].value_counts(dropna=False, normalize=True).head(50) *100)
df[['source of drinking water', 'time to get to water source (minutes)', 'location of source for water']].value_counts(dropna=False, normalize=True).head(50) *100

source of drinking water                            
public tap/standpipe                                    14.657350
tube well or borehole                                   14.249103
unprotected well                                        13.624185
piped into dwelling                                     11.699621
river/dam/lake/ponds/stream/canal/irrigation channel    11.263581
protected well                                           8.785387
piped to yard/plot                                       8.305744
unprotected spring                                       5.730952
protected spring                                         3.716805
piped to neighbor                                        2.272047
sachet water                                             1.493597
rainwater                                                1.196356
well in residence                                        1.164139
water seller                                             1.112171
other                  

source of drinking water                              time to get to water source (minutes)  location of source for water
piped into dwelling                                   0.0                                    in own dwelling                 11.687856
piped to yard/plot                                    0.0                                    in own yard/plot                 6.371142
public tap/standpipe                                  10.0                                   elsewhere                        2.833340
tube well or borehole                                 10.0                                   elsewhere                        2.270050
public tap/standpipe                                  5.0                                    elsewhere                        2.131359
unprotected well                                      10.0                                   elsewhere                        1.892023
tube well or borehole                                 30.0          

In [8]:
subdf = df[['source of drinking water', 'time to get to water source (minutes)', 'location of source for water']]
print(subdf.value_counts(dropna=False, normalize=True).head(50) *100)
subdf.loc[subdf['time to get to water source (minutes)'] >= 5.0, 'time to get to water source (minutes)'] = 5.0
print(subdf.value_counts(dropna=False, normalize=True).head(50) *100) 

source of drinking water                              time to get to water source (minutes)  location of source for water
piped into dwelling                                   0.0                                    in own dwelling                 11.687856
piped to yard/plot                                    0.0                                    in own yard/plot                 6.371142
public tap/standpipe                                  10.0                                   elsewhere                        2.833340
tube well or borehole                                 10.0                                   elsewhere                        2.270050
public tap/standpipe                                  5.0                                    elsewhere                        2.131359
unprotected well                                      10.0                                   elsewhere                        1.892023
tube well or borehole                                 30.0          

In [9]:
# Looks clean
water_treatment_cols = ['anything done to water to make safe to drink', 'water not available for at least a day last two weeks'] + [c for c in df.columns if 'treated' in c]
water_treatment_cols
for col in water_treatment_cols:
    print(df[col].value_counts(dropna=False, normalize=True).head(50) *100)

anything done to water to make safe to drink
no     46.319540
NaN    41.244645
yes    12.435815
Name: proportion, dtype: float64
water not available for at least a day last two weeks
NaN                                        88.949528
no, not interrupted for a full day          7.489680
yes, interrupted for a full day or more     3.560792
Name: proportion, dtype: float64
water usually treated by: boil
no     53.289049
NaN    41.247613
yes     5.463338
Name: proportion, dtype: float64
water usually treated by: add bleach/chlorine
no     54.469540
NaN    41.247613
yes     4.282847
Name: proportion, dtype: float64
water usually treated by: strain through a cloth
no     56.677422
NaN    41.382094
yes     1.940484
Name: proportion, dtype: float64
water usually treated by: let it stand and settle
no     57.800763
NaN    41.313720
yes     0.885516
Name: proportion, dtype: float64
water usually treated by: other
no     60.514570
NaN    39.330388
yes     0.155042
Name: proportion, dtype: float

In [10]:
df = df2
print(df[['type of toilet facility']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['share toilet with other households']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['number of households sharing toilet']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['location of toilet facility']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['share toilet with other households', 'number of households sharing toilet']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['type of toilet facility', 'location of toilet facility']].value_counts(dropna=False, normalize=True).head(50) *100)

print('-------------in-----------------')
# fill nas of share toilet with other households where type of toilet facility is no facility/bush/field
df.loc[(df['type of toilet facility'] == 'no facility/bush/field'), 'share toilet with other households'] = 'not applicable'
# df['share toilet with other households: applicable'] = np.where(df['type of toilet facility'] == 'no facility/bush/field', 'not applicable', 'applicable')
# df['share toilet with other households: not applicable'] = np.where(df['type of toilet facility'] == 'no facility/bush/field', 1.0, 0.0)

df.loc[(df['share toilet with other households'] == 'no'), 'number of households sharing toilet'] = 0.0
df.loc[(df['type of toilet facility'] == 'no facility/bush/field'), 'number of households sharing toilet'] = 10.0
df['number of households sharing toilet: applicable'] = np.where(df['type of toilet facility'] == 'no facility/bush/field', 'not applicable', 'applicable')
# df['number of households sharing toilet: not applicable'] = np.where(df['type of toilet facility'] == 'no facility/bush/field', 1.0, 0.0)


print(df[['share toilet with other households', 'number of households sharing toilet']].value_counts(dropna=False, normalize=True).head(50) *100)
df.loc[(df['type of toilet facility'] == 'no facility/bush/field') & (df['location of toilet facility'].isna()), 'location of toilet facility'] = 'elsewhere'

print(df[['type of toilet facility']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['share toilet with other households']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['number of households sharing toilet']].value_counts(dropna=False, normalize=True).head(50) *100)
print(df[['location of toilet facility']].value_counts(dropna=False, normalize=True).head(50) *100)

df[['type of toilet facility', 'share toilet with other households']].value_counts(dropna=False, normalize=True).head(50) *100

type of toilet facility              
no facility/bush/field                   25.875816
pit latrine with slab                    16.912991
pit latrine without slab/open pit        16.506633
traditional pit toilet                   15.325063
ventilated improved pit latrine (vip)     7.302313
flush to septic tank                      5.402951
flush to piped sewer system               3.822685
modern flush toilet                       3.577413
traditional bucket flush                  2.847802
flush to pit latrine                      1.679993
other flush                               0.562480
NaN                                       0.183860
Name: proportion, dtype: float64
share toilet with other households
no                                    40.456859
NaN                                   33.763814
yes                                   25.779326
Name: proportion, dtype: float64
number of households sharing toilet
NaN                                    78.031163
2.0                 

type of toilet facility                share toilet with other households
no facility/bush/field                 not applicable                        25.875816
pit latrine without slab/open pit      no                                     9.727945
pit latrine with slab                  no                                     8.859212
                                       yes                                    7.683416
pit latrine without slab/open pit      yes                                    6.526832
traditional pit toilet                 no                                     6.044868
                                       NaN                                    5.049748
                                       yes                                    4.230447
flush to septic tank                   no                                     3.647946
ventilated improved pit latrine (vip)  yes                                    3.329065
                                       no               

In [11]:
for col in ['type of cookstove', 'type of cooking fuel', 'food cooked in the house/ separate building/ outdoors', 'household has separate room used as kitchen']:
    print(df[col].value_counts(dropna=False, normalize=True).head(50) *100)

df['household has separate room used as kitchen'] = np.where(
    ((df['food cooked in the house/ separate building/ outdoors'] == 'in a seperate building') |
    (df['food cooked in the house/ separate building/ outdoors'] == 'outdoors')), 
    'not applicable', 
    df['household has separate room used as kitchen'])
print(df['household has separate room used as kitchen'].value_counts(dropna=False, normalize=True).head(50) *100)


type of cookstove
NaN                                                93.754489
three stone stove/open fire                         3.726033
liquefied petroleum gas (lpg)/cooking gas stove     1.158797
manufactured solid fuel stove                       0.678506
traditional solid fuel stove                        0.517689
no food cooked in household                         0.134913
electric stove                                      0.029573
Name: proportion, dtype: float64
type of cooking fuel
wood                       45.559709
charcoal                   13.839290
NaN                        13.250637
firewood, straw            11.405456
lpg                         8.815877
kerosene                    2.971221
electricity                 2.384132
no food cooked in house     1.042771
animal dung                 0.567715
other                       0.163137
96.0                        0.000054
Name: proportion, dtype: float64
food cooked in the house/ separate building/ outdoors
NaN    

In [12]:
for col in ['owns livestock, herds or farm animals', 'owns goats', 'owns horses/ donkeys/ mules', 'owns cows/ bulls', 'owns chickens/poultry', 'owns sheep']:
    print(df[col].value_counts(dropna=False, normalize=True).head(50) *100)
    if col != 'owns livestock, herds or farm animals':
        df[col] = np.where(((df['owns livestock, herds or farm animals'] == 'no') & (df[col].isna())), 0.0, df[col])
        print(df[col].value_counts(dropna=False, normalize=True).head(50) *100)

print(df[['owns livestock, herds or farm animals', 'owns goats']].value_counts(dropna=False, normalize=True).head(50) *100)

owns livestock, herds or farm animals
yes    36.052212
no     32.711923
NaN    31.235864
Name: proportion, dtype: float64
owns goats
0       50.292519
NaN     33.144077
2.0      3.031230
1.0      2.378196
3.0      2.240045
4.0      1.802279
5.0      1.591652
10.0     0.992368
6.0      0.988104
7.0      0.589625
8.0      0.496319
20.0     0.398803
15.0     0.326814
12.0     0.236422
9.0      0.221581
30.0     0.179111
11.0     0.108632
95.0     0.097839
13.0     0.092928
40.0     0.079599
25.0     0.074040
14.0     0.071504
50.0     0.069831
16.0     0.061790
18.0     0.046950
17.0     0.044252
23.0     0.028008
22.0     0.025580
19.0     0.022504
21.0     0.022288
35.0     0.021856
60.0     0.021856
24.0     0.016945
26.0     0.014301
28.0     0.013869
32.0     0.012304
27.0     0.012196
70.0     0.012088
45.0     0.011225
80.0     0.008958
34.0     0.008041
36.0     0.007231
29.0     0.007015
33.0     0.005936
31.0     0.005019
42.0     0.004911
38.0     0.004533
37.0     0.004317
90.

In [13]:
for col in ['owns land usable for agriculture', 'hectares of agricultural land (1 decimal)']:
    print(df[col].value_counts(dropna=False, normalize=True).head(50) *100)
df[['owns land usable for agriculture', 'hectares of agricultural land (1 decimal)']].value_counts(dropna=False, normalize=True).head(50) *100
df['hectares of agricultural land (1 decimal)'] = np.where(((df['owns land usable for agriculture'] == 'no') & (df['hectares of agricultural land (1 decimal)'].isna())), 
                                                           0.0, df['hectares of agricultural land (1 decimal)'])
df[['owns land usable for agriculture', 'hectares of agricultural land (1 decimal)']].value_counts(dropna=False, normalize=True).head(50) *100



owns land usable for agriculture
yes    40.545147
NaN    31.712216
no     27.742152
9.0     0.000486
Name: proportion, dtype: float64
hectares of agricultural land (1 decimal)
NaN           62.373512
10.0           3.731430
20.0           3.536183
1.0            2.680078
2.0            2.529731
don't know     2.417483
0.0            2.296709
4.0            2.253213
30.0           1.917279
5.0            1.892886
8.0            1.681882
40.0           1.496511
50.0           1.136563
3.0            1.017623
12.0           1.012551
15.0           0.742670
6.0            0.613369
100.0          0.582124
60.0           0.576889
16.0           0.569981
25.0           0.344083
95.0           0.327731
7.0            0.307009
80.0           0.305012
70.0           0.277490
24.0           0.248186
1.0            0.207874
200.0          0.183644
150.0          0.175657
32.0           0.143170
35.0           0.135291
28.0           0.129301
120.0          0.127574
9.0            0.094979
90.0    

owns land usable for agriculture  hectares of agricultural land (1 decimal)
NaN                               NaN                                          31.700560
no                                0.0                                          27.301957
yes                               10.0                                          3.731430
                                  20.0                                          3.536183
                                  NaN                                           3.370996
                                  1.0                                           2.680078
                                  2.0                                           2.529731
                                  don't know                                    2.405827
                                  4.0                                           2.253213
                                  30.0                                          1.917279
                                  

In [14]:
df[["type of place of residence", 'owns livestock, herds or farm animals']].value_counts(dropna=False, normalize=True).head(50) *100
# df[["type of place of residence", "place of residence"]]

type of place of residence  owns livestock, herds or farm animals
rural                       yes                                      29.490411
                            NaN                                      20.476103
urban                       no                                       17.561006
rural                       no                                       15.150917
urban                       NaN                                      10.759761
                            yes                                       6.561801
Name: proportion, dtype: float64

In [15]:
df[["type of place of residence", 'owns land usable for agriculture']].value_counts(dropna=False, normalize=True).head(50) *100

type of place of residence  owns land usable for agriculture
rural                       yes                                 33.674610
                            NaN                                 20.532605
urban                       no                                  16.832205
                            NaN                                 11.179611
rural                       no                                  10.909947
urban                       yes                                  6.870537
rural                       9.0                                  0.000270
urban                       9.0                                  0.000216
Name: proportion, dtype: float64

In [16]:
cols =['place where household members wash their hands', 'presence of water at hand washing place', 
                          'items present: soap or detergent', 'items present: ash, mud, sand', 'items present: none']
for nr, col in enumerate(cols):
    print(df[col].value_counts(dropna=False, normalize=True).head(50) *100)
    if col != 'place where household members wash their hands':
        print(df[['place where household members wash their hands', col]].value_counts(dropna=False, normalize=True).head(50) *100)
        df[col] = np.where((df['place where household members wash their hands'].isin(['not observed: not in dwelling', 'not observed other']) & df[col].isna()), 
                           'not applicable', df[col])
        print(df[['place where household members wash their hands', col]].value_counts(dropna=False, normalize=True).head(50) *100)


place where household members wash their hands
NaN                              58.373497
not observed: not in dwelling    13.583279
observed, mobile place           10.740226
observed                          7.310138
not observed other                5.556590
observed, fixed place             4.436271
Name: proportion, dtype: float64
presence of water at hand washing place
NaN                    77.524699
water is available     13.335686
water not available     9.139615
Name: proportion, dtype: float64
place where household members wash their hands  presence of water at hand washing place
NaN                                             NaN                                        58.373497
not observed: not in dwelling                   NaN                                        13.583279
not observed other                              NaN                                         5.556590
observed, mobile place                          water not available                         5.50262

In [17]:
df.to_pickle(out_f)
df.to_csv(out_f.replace('.pkl', '.csv'), index=False)